In [1]:
import torch
import pandas as pd
import numpy as np
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn
import torch.nn.functional as F
from torch import optim
from sklearn.model_selection import train_test_split

In [2]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

In [3]:
dataframe=np.genfromtxt("diabetes.csv",delimiter=',',skip_header=1)

In [4]:
X=dataframe[:,0:8]
Y=dataframe[:,8]
Y=Y.reshape(768,1)
dtype=torch.float
device=torch.device("cpu")

In [5]:
from sklearn.preprocessing import StandardScaler
model=StandardScaler()
X=model.fit_transform(X)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [7]:
data = torch.from_numpy(X_train)
target = torch.from_numpy(y_train)
data,target=data.type(torch.FloatTensor),target.type(torch.FloatTensor)

In [8]:
data_length,data_width=data.shape

In [9]:
bobs_data = data[0:int(data_length/2)].send(bob)
bobs_target = target[0:int(data_length/2)].send(bob)

alices_data = data[int(data_length/2):].send(alice)
alices_target = target[int(data_length/2):].send(alice)


In [10]:
# Iniitalize A Toy Model
model = nn.Linear(8,1)
loss= torch.nn.MSELoss()

In [11]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

In [12]:
for i in range(1000):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = loss(bobs_pred,bobs_target)
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = loss(alices_pred,alices_target)
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
#     print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

In [13]:
iterations = 10
worker_iters = 200

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model.forward(bobs_data)
        bobs_loss = loss(bobs_pred,bobs_target)
        bobs_loss.backward()        
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model.forward(alices_data)
        alices_loss = loss(alices_pred,alices_target)
        alices_loss.backward()
        alices_opt.step()
#         alices_opt.step(alices_data.shape[0])
        alices_loss = alices_loss.get().data
#         print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.5271) Alice:tensor(0.5277)
Bob:tensor(0.3143) Alice:tensor(0.3126)
Bob:tensor(0.2319) Alice:tensor(0.2298)
Bob:tensor(0.1963) Alice:tensor(0.1945)
Bob:tensor(0.1798) Alice:tensor(0.1783)
Bob:tensor(0.1716) Alice:tensor(0.1706)
Bob:tensor(0.1673) Alice:tensor(0.1668)
Bob:tensor(0.1649) Alice:tensor(0.1649)
Bob:tensor(0.1635) Alice:tensor(0.1639)
Bob:tensor(0.1626) Alice:tensor(0.1633)


In [14]:
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
X_test,y_test=X_test.type(torch.FloatTensor),y_test.type(torch.FloatTensor)
pred=model(X_test)

In [15]:
pred=torch.round(pred)
output=pred-y_test

In [16]:
incorrect_labels=len(np.nonzero(output))

accuracy=1-(incorrect_labels/len(X_test))

accuracy

0.8311688311688312

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.detach().numpy(),pred.detach().numpy()))